### 001 - Carregamento das variáveis de ambiente

In [3]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

### 002 - Importação das bibliotecas necessárias

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from pinecone import Pinecone, ServerlessSpec
from operator import itemgetter

/home/weriks/anaconda3/envs/sistema_turismo_langchain/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


### 003 - Definição dos modelos de linguagem

In [5]:
# model = ChatGroq(model='llama-3.1-8b-instant',temperature=0)
model = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct',temperature=0)

In [6]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipykernel_1096749/1048288091.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name=model_name)


### 004 - Configuração do Pinecone e preenchimento do index

In [7]:
pinecone_client = Pinecone()

# Define o index name do Pinecone
index_name = "sistema-turismo-langchain"
index_name_exists = False

# Verifica se o index name já existe
for index in pinecone_client.list_indexes():
  index_name = index['name']

  if index_name == "sistema-turismo-langchain":
    index_name_exists = True
    break

# Cria o index name caso não exista
if not index_name_exists:
  pinecone_client.create_index(
    name=index_name,
    metric="cosine",
    dimension=384,
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
  )

In [8]:
base_conhecimento_paris = "./base_conhecimento/paris.txt"
base_conhecimento_rio_de_janeiro = "./base_conhecimento/rio_de_janeiro.txt"

# Carrega os documentos de texto
loader_paris = TextLoader(base_conhecimento_paris)
loader_rio = TextLoader(base_conhecimento_rio_de_janeiro)

documents_paris = loader_paris.load()
documents_rio = loader_rio.load()

# Combina os documentos em uma única lista
all_documents = documents_paris + documents_rio

# Divide os documentos em chunks menores
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200)

docs_split = text_splitter.split_documents(all_documents)

In [9]:
# Cria o vectorstore no Pinecone
vectorstore = PineconeVectorStore.from_documents(
  documents=docs_split,
  embedding=embeddings_model,
  index_name=index_name
)

In [10]:
# Cria o retriever para buscar os documentos mais similares
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

### 005 - Definição dos prompt templates

##### Guia geral para alguns templates

In [11]:
guide_to_the_assistants = """
  Sua tarefa é responder à consulta do usuário baseando-se ÚNICA E EXCLUSIVAMENTE no contexto fornecido.
  É estritamente proibido utilizar qualquer conhecimento externo ao que foi apresentado no contexto.
  Não invente informações. Não complete detalhes que não estão no texto.
  Se a resposta para a pergunta não puder ser encontrada no contexto fornecido, você deve responder exatamente:
  "Desculpe, não tenho informações sobre isso na minha base de conhecimento atual."
  Não tente adivinhar ou inferir informações.
"""

##### Prompt para categorizar a intenção do usuário

In [12]:
# Recebe a intenção do usuário e classifica em uma das categorias predefinidas
prompt_template_category_definition = PromptTemplate.from_template(
  """
    Classifique a pergunta do usuário em uma das seguintes categorias. Seja muito severo na classificação.
    Escreva apenas uma categoria. Por exemplo, se a pergunta for sobre roteiro de viagem, responda apenas "Roteiro de Viagem".
    Se a pergunta não se enquadrar em nenhuma das categorias, responda apenas "Geral".

    Categorias:
    - Roteiro de Viagem
    - Logística de Transporte
    - Informações sobre pontos turísticos
    - Guia de tradução e idiomas
    - Geral

    Pergunta: {query}
    Classificação:
  """
)

category_classification_chain = (prompt_template_category_definition | model | StrOutputParser())

##### Prompt para extrair o perfil do usuário

In [13]:
# Cadeia para extrair o perfil do usuário
prompt_extract_profile = PromptTemplate.from_template(
"""
  Analise a consulta do usuário e identifique o perfil do turista.
  Se não for possível identificar, retorne "geral".

  Perfis possíveis: aventura, cultural, gastronômico, relaxante, família.

  Consulta do usuário: {query}
  Perfil:
"""
)

profile_extraction_chain = prompt_extract_profile | model | StrOutputParser()

##### Prompt para responder perguntas sobre os destinos turísticos

In [14]:
prompt_template_travel_itinerary = PromptTemplate(
  input_variables=["query", "profile", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer roteiros de viagem personalizados.
    Com base na consulta do usuário, crie um roteiro detalhado que inclua:
    - Principais atrações a serem visitadas
    - Eventos locais durante o período da viagem
    - Sugestões que se alinhem ao perfil do turista.

    Perfil do turista: {profile}
    Consulta do usuário: {query}
    Context: {context}
  """ % guide_to_the_assistants
)

##### Prompt para logisticas de transporte

In [15]:
prompt_template_transport_logistics = PromptTemplate(
  input_variables=["query", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer informações sobre logística de transporte.
    Com base na consulta do usuário, apresente um relatório logístico que inclua:
    - Opções de transporte do local
    - Sugestões de acomodação
    - Dicas práticas para facilitar a viagem

    Consulta do usuário: {query}
    Context: {context}

  """ % guide_to_the_assistants
)

##### Prompt para responder perguntas sobre informações locais

In [16]:
prompt_template_local_information = PromptTemplate(
  input_variables=["query", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer informações locais detalhadas.
    Com base na consulta do usuário, forneça informações específicas sobre:
    - Pontos turísticos
    - Restaurantes recomendados
    - Horários de funcionamento e outras informações relevantes

    Consulta do usuário: {query}
    Context: {context}
    """ % guide_to_the_assistants
)

##### Prompt para responder perguntas sobre tradução de frases

In [17]:
prompt_template_translation_guide = PromptTemplate(
  input_variables=["query"],
  template= """
    Você é um assistente especializado em fornecer guias de tradução para viajantes.
    Com base na consulta do usuário, crie um guia de tradução que inclua:
    - Frases úteis no idioma local
    - Dicas culturais para facilitar a comunicação

    Consulta do usuário: {query}
    """
)

##### Prompt para responder perguntas sobre informações gerais

In [18]:
prompt_template_general = PromptTemplate(
  input_variables=["query"],
  template="""
    Guia geral: %s

    Você é um assistente de informações gerais.
    Se a consulta do usuário não se enquadrar nas categorias de roteiro de viagem,
    logística de transporte, informações locais ou guia de tradução,
    responda com uma mensagem educada informando que a consulta não pode ser atendida.
    E sugira tipos de perguntas que você pode responder.

    Consulta do usuário: {query}
    """ % (guide_to_the_assistants)
)

### 006 - Definição das chains

In [19]:
# Define chain para itinerário de viagem
travel_itinerary_chain = {
    "profile": itemgetter("query") | profile_extraction_chain,
    "context": itemgetter("query") | retriever,
    "query": itemgetter("query")
} | prompt_template_travel_itinerary

# Define chain para logística de transporte
transport_logistics_chain = {
    "context": itemgetter("query") | retriever,
    "query": itemgetter("query")
} | prompt_template_transport_logistics

# Define chain para informações locais
local_information_chain = {
    "context": itemgetter("query") | retriever,
    "query": itemgetter("query")
} | prompt_template_local_information

# Define a cadeia completa com branching
full_chain = RunnableBranch(
    (
        # Se a categoria for "Roteiro de Viagem", use a cadeia de itinerário de viagem
        lambda x: "roteiro de viagem" in x["category"].lower(),
        travel_itinerary_chain
    ),
    (
        # Se a categoria for "Logística de Transporte", use a cadeia de logística de transporte
        lambda x: "logística de transporte" in x["category"].lower(),
        transport_logistics_chain
    ),
    (
        # Se a categoria for "Informações sobre pontos turísticos", use a cadeia de informações locais
        lambda x: "informações sobre pontos turísticos" in x["category"].lower(),
        local_information_chain
    ),
    (
        # Se a categoria for "Guia de tradução e idiomas", use a cadeia de guia de tradução
         lambda x: "guia de tradução" in x["category"].lower(),
         itemgetter("query") | prompt_template_translation_guide
    ),
    # Cadeia padrão para consultas gerais ou não classificadas
    itemgetter("query") | prompt_template_general
)

# Cadeia final que inclui a classificação de categoria e a cadeia completa
# 1. Descobre a categoria da consulta do usuário
# 2. Roteia para a cadeia específica baseada na categoria (full_chain)
# 3. O resultado é passado para o modelo para gerar a resposta final
# 4. A resposta final é formatada como string
final_orchestrator_chain = {
    "category": category_classification_chain,
    "query": itemgetter("query")
} | full_chain| model | StrOutputParser()

### 007 - Execução de exemplos

In [20]:
# Exemplo 1: Roteiro de Viagem
query = "Estou planejando uma viagem gastronomica de 3 dias em Paris. Quais atrações você recomenda?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

Claro, posso ajudar a criar um roteiro de viagem gastronômica para Paris! Embora o contexto fornecido não tenha informações específicas sobre atrações gastronômicas, posso sugerir alguns pontos turísticos e bairros que podem ser interessantes para um turista gastronômico com base nas informações disponíveis.

### Roteiro de Viagem Gastronômica para Paris

#### Dia 1:
- **Manhã:** Visite o **Mercado de Noël sous les arbres** (se estiver viajando em dezembro) ou explore o **Le Marais** (4º arrondissement), conhecido por sua vibe boêmia e opções gastronômicas.
- **Tarde:** Explore o **Museu do Louvre** (1º arrondissement). Embora não seja uma atração gastronômica per se, o Louvre fica em um bairro com muitas opções de restaurantes.
- **Noite:** Jantar no **5º ou 6º arrondissement (Latin Quarter e Saint-Germain)**, conhecidos por sua gastronomia.

#### Dia 2:
- **Manhã:** Café da manhã em um café clássico parisiense no **1º ou 2º arrondissement**, perto do Louvre.
- **Tarde:** Visite o **C

In [21]:
# Exemplo 2: Logística de Transporte
query = "Quais são as melhores opções de transporte do aeroporto Charles de Gaulle para o centro de Paris?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

## Relatório Logístico: Transporte do Aeroporto Charles de Gaulle para o Centro de Paris

### Opções de Transporte

1. **Trem RER B:** 
   - O trem RER B é uma das opções mais rápidas e eficientes para se deslocar do Aeroporto Charles de Gaulle para o centro de Paris.
   - O tempo de viagem varia de acordo com a estação de destino, mas geralmente leva entre 35 a 45 minutos até o centro.
   - As estações RER B no centro de Paris incluem Denfert-Rochereau, Saint-Michel Notre-Dame, e Gare du Nord, entre outras.

2. **Metrô (futuro):** 
   - Em 2025, novas linhas de metrô automatizadas estarão operacionais, reduzindo ainda mais o tempo de viagem entre o Aeroporto Charles de Gaulle e o centro de Paris.

### Sugestões de Acomodação

Embora o contexto fornecido não detalhe opções específicas de acomodação, é recomendável considerar hotéis ou acomodações próximas às estações de metrô ou RER para facilitar o deslocamento. Algumas áreas centrais para se hospedar incluem:

- **4º arrondissement:*

In [22]:
# Exemplo 3: Informações sobre pontos turísticos
query = "Quais são os pontos turisticos de paris?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

Os principais pontos turísticos de Paris incluem:

1. **Torre Eiffel**: O monumento mais emblemático de Paris, construído por Gustave Eiffel para a Exposição Universal de 1889. Oferece vistas incríveis da cidade a partir de seus três níveis.
   - Localização: Champ de Mars, 7º arrondissement.
   - Como Chegar: Metrô (estação Bir-Hakeim, linha 6, ou Trocadéro, linhas 6 e 9 para a melhor vista).
   - Horário de Funcionamento: Geralmente das 9h30 às 23h.

2. **Arco do Triunfo**: Monumento histórico que homenageia os soldados franceses, com vistas do topo dos Champs-Élysées e da cidade.
   - Localização: Place Charles de Gaulle, 8º arrondissement.
   - Como Chegar: Metrô (estação Charles de Gaulle - Étoile, linhas 1, 2 e 6).

3. **Museu d'Orsay**: Museu em uma antiga estação de trem, focado em arte impressionista (Monet, Van Gogh).
   - Localização: 7º arrondissement.
   - Como Chegar: Metrô (estação Musée d'Orsay, RER C).

Além disso, Paris é conhecida por seus monumentos icônicos, museus

In [23]:
# Exemplo 4: Guia de tradução e idiomas
query = "Como digo 'onde fica o banheiro?' em francês?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

## Guia de Tradução para Falantes de Português em Países Francófonos

### Introdução

Ao viajar para países onde o francês é a língua oficial, como a França, o Canadá (em algumas províncias), a Bélgica, a Suíça, entre outros, é útil saber algumas frases básicas para se comunicar de forma eficaz. Este guia foca em fornecer frases úteis e dicas culturais para ajudar você a se sentir mais confortável e confiante durante sua viagem.

### Frases Úteis em Francês

#### Perguntas Básicas

- **Onde fica o banheiro?** - "Où sont les toilettes?" (Pronúncia: "u são le twaletes?")
- **Onde posso encontrar...?** - "Où puis-je trouver...?" (Pronúncia: "u pwiz-je truver...")
- **Quanto custa?** - "Combien ça coûte?" (Pronúncia: "kombien sa cout?")

#### Saudações

- **Olá** - "Bonjour" (Pronúncia: "bonjour")
- **Tudo bem?** - "Comment allez-vous?" (Pronúncia: "koman alé vu?")
- **Obrigado/Obrigada** - "Merci" (Pronúncia: "mersi")
- **Por favor** - "S'il vous plaît" (Pronúncia: "sil vu ple")
- **Descu

In [24]:
# Exemplo 5: Geral
query = "Qual é a capital da França?"
final_response = final_orchestrator_chain.invoke({"query": query})
print(final_response)

Desculpe, não tenho informações sobre isso na minha base de conhecimento atual. No entanto, posso ajudar com perguntas relacionadas a roteiros de viagem, logística de transporte, informações locais ou guias de tradução. Por favor, sinta-se à vontade para perguntar sobre esses tópicos.
